In [1]:
import pandas as pd
import requests
import json
import datetime
import csv
import time
import matplotlib.pyplot as plt
import ast

In [2]:
TOKEN = "ee3oyaYzlZrYuCC0611Oezn0N"
base_url = "https://data.cityofnewyork.us/resource/enfh-gkve.json"

headers = {"X-App-Token": TOKEN}

limit = 1000    
offset = 0
all_records = []

while True:
    url = f"{base_url}?$limit={limit}&$offset={offset}"
    try:
        response = requests.get(url, headers=headers, timeout=20)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print("Request failed, retrying in 2 seconds...", e)
        time.sleep(2)
        continue

    batch = response.json()

    if not batch:
        print("No more data returned. Stopping.")
        break

    all_records.extend(batch)
    print(f"Fetched {len(batch)} rows (offset={offset})")

    # Stop if fewer than the limit means end of dataset
    if len(batch) < limit:
        break

    offset += limit
    time.sleep(0.2)  # polite rate-limit protection


# dataframe:

parks = pd.DataFrame(all_records)

print("Done! Total rows:", len(parks))
print(parks.head())

Fetched 1000 rows (offset=0)
Fetched 1000 rows (offset=1000)
Fetched 55 rows (offset=2000)
Done! Total rows: 2055
           acquisitiondate    acres                address borough class  \
0  1972-09-22T00:00:00.000     83.3  1000 RICHMOND TERRACE       R  PARK   
1                      NaN    0.356   200-15 MCLAUGHLIN AV       Q  PARK   
2  1926-04-29T00:00:00.000  286.557    298 SATERLEE STREET       R  PARK   
3  1990-10-26T00:00:00.000    6.611          2 KENT STREET       B  PARK   
4  1948-01-29T00:00:00.000    1.534   255 PARKINSON AVENUE       R  PARK   

  communityboard councildistrict department                       eapply  \
0            501              49       R-01  Snug Harbor Cultural Center   
1            408              23       Q-08        McLaughlin Playground   
2            503              51       R-03        Conference House Park   
3            301              33       B-01        WNYC Transmitter Park   
4            502              50      R-02B      

In [3]:
columns_to_keep = ["typecategory", "name311", "acres", "multipolygon", "waterfront"]

In [4]:
parks = parks[columns_to_keep]

In [5]:
parks.head()

,typecategory,name311,acres,multipolygon,waterfront
0,Historic House Park,Snug Harbor Cultural Center,83.3,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",True
1,Playground,McLaughlin Playground,0.356,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",False
2,Nature Area,Conference House Park,286.557,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",True
3,Neighborhood Park,WNYC Transmitter Park,6.611,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",True
4,Playground,Old Town Playground,1.534,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",False


In [7]:
import pandas as pd

def compute_centroid_safe(multipolygon):
    # If it's a string, convert to dict
    if isinstance(multipolygon, str):
        import ast
        multipolygon = ast.literal_eval(multipolygon)
    
    coords = multipolygon['coordinates']
    
    # Flatten all coordinate points (handles multiple polygons and rings)
    all_points = []
    for polygon in coords:
        for ring in polygon:
            all_points.extend(ring)  # ring is a list of [lon, lat] pairs
    
    # Compute average longitude and latitude
    avg_lon = sum(pt[0] for pt in all_points) / len(all_points)
    avg_lat = sum(pt[1] for pt in all_points) / len(all_points)
    
    return avg_lon, avg_lat

# Apply to DataFrame
parks['centroid'] = parks['multipolygon'].apply(compute_centroid_safe)

# Optionally split into lon/lat columns
parks[['centroid_lon', 'centroid_lat']] = pd.DataFrame(parks['centroid'].tolist(), index=parks.index)

print(parks[['centroid_lon', 'centroid_lat']].head())


   centroid_lon  centroid_lat
0    -74.103130     40.644863
1    -73.766289     40.725611
2    -74.239806     40.501085
3    -73.961302     40.729879
4    -74.081184     40.595311


In [8]:
parks.head()

,typecategory,name311,acres,multipolygon,waterfront,centroid,centroid_lon,centroid_lat
0,Historic House Park,Snug Harbor Cultural Center,83.3,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",True,"(-74.10312983893252, 40.644863105489236)",-74.103130,40.644863
1,Playground,McLaughlin Playground,0.356,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",False,"(-73.76628896333246, 40.72561067380002)",-73.766289,40.725611
2,Nature Area,Conference House Park,286.557,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",True,"(-74.23980570077656, 40.50108512752425)",-74.239806,40.501085
3,Neighborhood Park,WNYC Transmitter Park,6.611,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",True,"(-73.96130246175792, 40.72987914861486)",-73.961302,40.729879
4,Playground,Old Town Playground,1.534,"{'type': 'MultiPolygon', 'coordinates': [[[[-7...",False,"(-74.08118377477349, 40.59531115962469)",-74.081184,40.595311


In [9]:
parks = parks.drop('multipolygon', axis=1)

In [10]:
parks.head()

,typecategory,name311,acres,waterfront,centroid,centroid_lon,centroid_lat
0,Historic House Park,Snug Harbor Cultural Center,83.3,True,"(-74.10312983893252, 40.644863105489236)",-74.103130,40.644863
1,Playground,McLaughlin Playground,0.356,False,"(-73.76628896333246, 40.72561067380002)",-73.766289,40.725611
2,Nature Area,Conference House Park,286.557,True,"(-74.23980570077656, 40.50108512752425)",-74.239806,40.501085
3,Neighborhood Park,WNYC Transmitter Park,6.611,True,"(-73.96130246175792, 40.72987914861486)",-73.961302,40.729879
4,Playground,Old Town Playground,1.534,False,"(-74.08118377477349, 40.59531115962469)",-74.081184,40.595311


In [11]:
len(parks)

2055

In [12]:
parks['typecategory'].value_counts()

typecategory
Triangle/Plaza                 361
Garden                         355
Neighborhood Park              295
Jointly Operated Playground    259
Playground                     254
Community Park                 109
Nature Area                     88
Recreational Field/Courts       71
Strip                           68
Undeveloped                     34
Mall                            33
Parkway                         30
Buildings/Institutions          21
Waterfront Facility             18
Managed Sites                   15
Flagship Park                   14
Cemetery                        12
Historic House Park             11
Lot                              5
Operations                       2
Name: count, dtype: int64